In [1]:
import os
import cv2
import numpy as np
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Dropout, Activation, Lambda, Cropping2D

CONFIG = {
    'batchsize': 32,
    'input_width': 320,
    'input_height': 160,
    'input_channels': 3,
    'correction': 0.2,
    'cropping': ((60,25), (0,0))
}

def load_and_split_data(data_path, test_size=0.2):
    with open(data_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        samples = [line for line in reader][1:]
    train_samples, validation_samples = train_test_split(samples, test_size=test_size, random_state=0)
    return train_samples, validation_samples

def generator(samples, img_path, side_camera=False, augment_data=False, batch_size=32):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                center, left, right, steering, throttle, brake, speed = batch_sample
                
                steering_center = float(steering)
                center_image = cv2.imread(os.path.join(img_path, center.strip()))
                
                images.append(center_image)
                angles.append(steering_center)
                
                if side_camera:
                    steering_left = steering_center + CONFIG['correction']
                    left_image = cv2.imread(os.path.join(img_path, left.strip()))
                    images.append(left_image)
                    angles.append(steering_left)

                    steering_right = steering_center - CONFIG['correction']
                    right_image = cv2.imread(os.path.join(img_path, right.strip()))
                    images.append(right_image)
                    angles.append(steering_right)
                        
            if augment_data:
                images_copy = images.copy()
                angles_copy = angles.copy()
                for image, angle in zip(images_copy, angles_copy):
                    images.append(cv2.flip(image, 1))
                    angles.append(angle*-1.0)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
def nvidia_model(summary=True):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(CONFIG['input_height'], CONFIG['input_width'], CONFIG['input_channels'])))
    model.add(Cropping2D(cropping=CONFIG['cropping']))
    model.add(Convolution2D(24,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dropout(0.5))
    model.add(Dense(50))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Dense(1))

    if summary:
        model.summary()
    
    return model

if __name__ == "__main__":
    train_samples, validation_samples = load_and_split_data(data_path='./data/data/driving_log.csv', test_size=0.2)

    train_generator = generator(train_samples, img_path='./data/data', side_camera=True, augment_data=True,  batch_size=CONFIG['batchsize'])
    validation_generator = generator(validation_samples, img_path='./data/data', batch_size=CONFIG['batchsize'])

    model = nvidia_model(summary=False)
    model.compile(optimizer='adam', loss='mse')

    model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*3, 
                                        validation_data=validation_generator, nb_val_samples=len(validation_samples), 
                                        nb_epoch=6, verbose=1)

    model.save('model3.h5')


Using TensorFlow backend.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Epoch 1/6
19200/19284 [============================>.] - ETA: 0s - loss: 0.0227

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


19392/19284 [==============================] - 40s - loss: 0.0228 - val_loss: 0.0131
Epoch 2/6
19368/19284 [==============================] - 38s - loss: 0.0191 - val_loss: 0.0121
Epoch 3/6
19392/19284 [==============================] - 37s - loss: 0.0174 - val_loss: 0.0124
Epoch 4/6
19368/19284 [==============================] - 37s - loss: 0.0176 - val_loss: 0.0124
Epoch 5/6
19392/19284 [==============================] - 36s - loss: 0.0163 - val_loss: 0.0123
Epoch 6/6
19368/19284 [==============================] - 36s - loss: 0.0164 - val_loss: 0.0118


In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3414915763188192412
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 204472320
locality {
  bus_id: 1
}
incarnation: 12352553770772824987
physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 67108864
locality {
  bus_id: 1
}
incarnation: 1794417282979895103
physical_device_desc: "device: 1, name: GRID K520, pci bus id: 0000:00:04.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 67108864
locality {
  bus_id: 1
}
incarnation: 16401946691897159913
physical_device_desc: "device: 2, name: GRID K520, pci bus id: 0000:00:05.0"
, name: "/gpu:3"
device_type: "GPU"
memory_limit: 67108864
locality {
  bus_id: 1
}
incarnation: 12109443125900092881
physical_device_desc: "device: 3, name: GRID K520, pci bus id: 0000:00:06.0"
]


In [2]:
!nvidia-smi

Mon Mar 19 21:33:58 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   66C    P0    57W / 125W |   4017MiB /  4036MiB |     84%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GRID K520           Off  | 0000:00:04.0     Off |                  N/A |
| N/A   50C    P8    17W / 125W |   3837MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G